In [16]:
## Defining a class
class NewCallback(tf.keras.callbacks.Callback):
    
    # defining sparsity
    def __init__(self, sparsity):
        self.sparsity=sparsity/100
        
    # main Function 
    def on_epoch_end(self,epochs,cd):
        # Function to find threshold
        def threshold_find(sparsity,weights):
            index=int(sparsity*weights.shape[0])
            threshold=(sorted(weights))[index]
            return threshold
        
        check=False
        for layers in reversed(self.model.layers):
            a=self.model.get_layer(layers.name)    
            e=a.get_weights()    # Taking layer wieghts
            
            if len(e)>0 and check == False: # to exclude layers with zero parameters and to skip the last layer
                check=True
                continue;
                
            if check==True and len(e)>0: 
                
                if 'lstm' in layers.name : # checking layer name
                    avg =np.average(e[1]**2,axis=0)+np.average(e[0]**2,axis=0) # avg there weights
                    weights1=0
                    units = e[1].shape[0]
                    
                    for i in range(0,4):
                        weights1=weights1+avg[units*i:units*(i+1)] # because of 4 gates
                    squared_sum =weights1/4
                    threshold = threshold_find(self.sparsity,squared_sum) # finding threshold
                    upper_threshold = (squared_sum > threshold)
                    
                    for i in range(0,4): # zeroing values below threshold
                        e[0][:,units*i:units*(i+1)]= e[0][:,units*i:units*(i+1)]*upper_threshold
                        e[1][:,units*i:units*(i+1)]= e[1][:,units*i:units*(i+1)]*upper_threshold
                        e[2][units*i:units*(i+1)]= e[2][units*i:units*(i+1)]*upper_threshold
                    a.set_weights(e) # setting weights back
                
                # repeating for dense and conv2d layer
                    
                if 'dense' in layers.name or 'conv2d' in layers.name or 'conv1d' in layers.name:
                    avg = np.average(e[0]**2,axis=tuple(np.arange(0,len(e[0].shape)-1)))
                    threshold = threshold_find(self.sparsity,avg)
                    upper_threshold = (avg > threshold)
                    e[0]=e[0]*upper_threshold
                    e[1]=e[1]*upper_threshold
                    a.set_weights(e)

In [1]:
# Final compression
delete_ind=[]
layer_details={}
allow=0
check_flatten = False
dense_flatten = np.array(None)
from keras.models import model_from_json
import json

prunned_model = pruned_model.to_json() # converting model into json
res = json.loads(prunned_model) 
i=0

for layers in pruned_model.layers:
    a=pruned_model.get_layer(layers.name) # getting weights
    e=a.get_weights()
    
    if 'Flatten' in str(layers):
        check_flatten= True
        
    #checking for layers with none parameters
    if len(e)>0:
        if 'lstm' in str(layers):
            e[0] = np.delete(e[0],delete_ind,0)   # removing dimension from previous layers 
            
            avg =np.average(e[1]**2,axis=0)+np.average(e[0]**2,axis=0)  # avg to find which filters to remove
            weights1=0
            units = e[1].shape[0]
            
            for j in range(0,4):
                weights1=weights1+avg[units*j:units*(j+1)]
            delete_ind = np.nonzero(weights1==0)
            
            
            #removing weights 
            c= np.concatenate((delete_ind[0],delete_ind[0]+units,delete_ind[0]+units*2,delete_ind[0]+units*3))
            e[0] = np.delete(e[0],c,1)
            e[1] = np.delete(e[1],delete_ind,0)
            e[1] = np.delete(e[1],c,1)
            e[2] = np.delete(e[2],c,0)
            
            
            # changing number of filters in model
            if (layers.name==res['config']['layers'][i]['name']):
                res['config']['layers'][i]['config']['units']=e[1].shape[0]
                
                
            # repeating for the dense layer 
        if 'Dense' in str(layers):
            if check_flatten == False:
                e[0] = np.delete(e[0],delete_ind,(len(e[0].shape)-2))
            else:
                for k in range(0,shape[1]*shape[2]+1):
                    dense_flatten = np.append(dense_flatten,delete_ind[0]+512*k)
                    delete_ind = tuple(dense_flatten[1:])
                    e[0] = np.delete(e[0],delete_ind,(len(e[0].shape)-2))
                check_flatten = False
            avg = np.average(e[0]**2,axis=tuple(np.arange(0,len(e[0].shape)-1)))
            delete_ind = np.nonzero(avg==0)
            e[0] = np.delete(e[0],delete_ind,(len(e[0].shape)-1))
            e[1] = np.delete(e[1],delete_ind,0)
            if (layers.name==res['config']['layers'][i]['name']):
                if 'Dense' in str(layers):
                    res['config']['layers'][i]['config']['units']=e[1].shape[0]
                    
            
            # repeat for conv layer
        if 'Conv2D' in str(layers) or 'Conv1D' in str(layers):
            e[0] = np.delete(e[0],delete_ind,(len(e[0].shape)-2))
            avg = np.average(e[0]**2,axis=tuple(np.arange(0,len(e[0].shape)-1)))
            delete_ind = np.nonzero(avg==0)
            e[0] = np.delete(e[0],delete_ind,(len(e[0].shape)-1))
            e[1] = np.delete(e[1],delete_ind,0)
            if (layers.name==res['config']['layers'][i]['name']):
                res['config']['layers'][i]['config']['filters']=e[1].shape[0]
                
    
        layer_details[layers.name] = {'weights':e}
    try:
        if check_flatten==False:
            shape=layers.output_shape
    except:
        pass
    
    res['config']['layers'][i]['config']['dtype']='float16' # quantization
    i=i+1
res=json.dumps(res)
prunned_model = tf.keras.models.model_from_json(res) # converting back model from json to model

# setting weights back
for layers in prunned_model.layers:
    if 'lstm' in str(layers) or 'Dense' in str(layers) or 'Conv2D' in str(layers) or 'Conv1D' in str(layers):
        a=prunned_model.get_layer(layers.name)
        e=a.get_weights()
        e=layer_details[layers.name]['weights']
        a.set_weights(e)        
        
prunned_model.summary()

IndentationError: expected an indented block (<ipython-input-1-feec1d583c0e>, line 82)

In [ ]:
model.fit(train_x,train_label,epochs=1,validation_split=0.05,batch_size=64,callbacks=[MyCustomCallback(#provide sparsity in %)])